In [9]:
import os
import numpy as np
from PIL import Image
import time
import tensorflow as tf
from ZeroDCE.zero_dce import ZeroDCE  # pas dit aan aan jouw bestandsstructuur

# 🔁 Preprocessing
def preprocess(image_path, input_shape):
    print(input_shape)
    height, width = input_shape  # input_shape is (H, W)
    image = Image.open(image_path).convert("RGB").resize((width, height))
    input_data = np.asarray(image, dtype=np.float32) / 255.0
    input_data = np.expand_dims(input_data, axis=0)
    return input_data, image

# 🔁 Inference
def enhance_image(model, input_data):
    start = time.time()
    output_data = model.predict(input_data)
    duration = time.time() - start
    print(f"Inference tijd: {duration:.2f} sec")

    print("Output shape:", output_data.shape)  # 👈 debug output

    output_image = np.squeeze(output_data)  # vorm: (H, W, 3) of (1, H, W, 3)
    
    if output_image.ndim == 3:
        pass  # alles ok
    elif output_image.ndim == 2:
        output_image = np.stack([output_image]*3, axis=-1)  # grijs → RGB
    else:
        raise ValueError(f"Onverwachte output shape: {output_image.shape}")

    output_image = output_image * 255
    output_image = np.clip(output_image, 0, 255).astype(np.uint8)
    return Image.fromarray(output_image)


# 📦 Laad het .h5 model
model = ZeroDCE(filters=32, iteration=6)  # zorg dat dit matcht met het oorspronkelijke model
dummy_input = tf.random.normal([1, 512, 384, 3])  # pas aan op input shape
_ = model(dummy_input, training=False)
model.load_weights("finetuned_models/zero_dce_finetune2/finetuned100epochs.h5")
input_shape = dummy_input.shape[1:3] # (H, W)


# 📁 Input/output mappen
base_input_folder = "../data-filtered-rooms"
base_output_folder = "../data-zerodce-finetuned100"
os.makedirs(base_output_folder, exist_ok=True)

for room_num in range(1, 32):
    room_name = f"room{room_num}"
    input_folder = os.path.join(base_input_folder, room_name)
    output_folder = os.path.join(base_output_folder, room_name)

    if not os.path.isdir(input_folder):
        print(f"Map {input_folder} bestaat niet, overslaan...")
        continue

    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            input_path = os.path.join(input_folder, filename)

            input_tensor, _ = preprocess(input_path, input_shape)
            #input_tensor, _ = preprocess(input_path, input_shape)
            enhanced_img = enhance_image(model, input_tensor)

            base_name = os.path.splitext(filename)[0]
            output_path = os.path.join(output_folder, base_name + ".png")
            enhanced_img.save(output_path)
            print(f"✅ Verwerkt: {input_path} → {output_path}")


(512, 384)
1/1 [==============================] - 0s 219ms/step
Inference tijd: 0.26 sec


AttributeError: 'tuple' object has no attribute 'shape'